<a id='top'></a>
<h1 style="text-align:center;font-size:200%;;">Transformers, BERT and GPT</h1>

---
---

# Introduction:<br>
### Welcome! In this demonstration we are going to understand the intuition behind Transformers and after that we are going to explore Google's BERT and Open AI's GPT.


## Table of Contents:<br>
1. Intution behind RNN based Sequence-to-Sequence Model<br>
   1.1 Limitations of RNN'S <br>
2. Introduction to the Transformers <br>
   2.1 Transformer’s Model Architecture<br>
   2.2 Understanding self attention<br>
   2.3 Limitations of the Transformer<br>
3. Understanding BERT <br>
   3.1 How Does BERT Work?<br>
   3.2 Text processing for BERT<br>
   3.3 How to use BERT (Fine-tuning)<br>
   3.4 Takeaways<br>
   3.5 Compute considerations (training and applying)<br>
   3.6 Tokenization<br>
   3.7 Sequence Classification<br>
   
   

### References and credits:
1. [BERT Explained: State of the art language model for NLP](https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270)<br>
2. [How do Transformers Work in NLP? A Guide to the Latest State-of-the-Art Models](https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/)<br>
3. [Demystifying BERT: A Comprehensive Guide to the Groundbreaking NLP Framework](https://www.analyticsvidhya.com/blog/2019/09/demystifying-bert-groundbreaking-nlp-framework/)
4. [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)<br>
5. [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)<br>
6. [Disaster NLP: Keras BERT using TFHub](https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub)<br>
Thanks to all the above blogs and article contributors.Thanks you very much for you open source contibutions.

# 1. Intution behind RNN based Sequence-to-Sequence Model:
Sequence-to-sequence (seq2seq) models are a type of neural network architecture that is commonly used for natural language processing (NLP) tasks such as language translation, speech recognition, and text summarization. The goal of a seq2seq model is to map an input sequence of tokens A to an output sequence of tokens B.

The architecture of a seq2seq model typically consists of two parts: an encoder and a decoder. The encoder takes in an input sequence of tokens and converts it into a fixed-length representation (also known as a context vector or latent representation) that captures the important information in the input sequence. The decoder then takes this context vector and generates the output sequence one token at a time, conditioned on the previous tokens it has generated.

One of the key advantages of seq2seq models is that they can handle variable-length input and output sequences. This makes them well-suited for tasks like machine translation, where the length of the input sentence can vary widely. Another advantage is that they can capture long-term dependencies in the input sequence, which is important for tasks like speech recognition where the current word may depend on words spoken several seconds ago.
Let’s take a simple example of a sequence-to-sequence model. Check out the below illustration:<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/seq2seq.gif)<br>
The above seq2seq model is converting a German phrase to its English counterpart. Let’s break it down:<br>

* Both Encoder and Decoder are RNNs
* At every time step in the Encoder, the RNN takes a word vector (xi) from the input sequence and a hidden state (Hi) from the previous time step
* The hidden state is updated at each time step
* The hidden state from the last unit is known as the context vector. This contains information about the input sequence
* This context vector is then passed to the decoder and it is then used to generate the target sequence (English phrase)
* If we use the Attention mechanism, then the weighted sum of the hidden states are passed as the context vector to the decoder

# 1.1 Limitations of RNN'S:
* Dealing with long-range dependencies is still challenging
* The sequential nature of the model architecture prevents parallelization.
* Vanishing Gradient Problem
* Issue of increasing gradients at each step called as exploding gradients.

# 2. Introduction to the Transformer:<br>
The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution. The idea behind Transformer is to handle the dependencies between input and output with attention and recurrence completely.<br>
## 2.1 Transformer’s Model Architecture:<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-19-53-10.png) <br>
Let’s first focus on the Encoder and Decoder parts only.Now focus on the below image. The Encoder block has 1 layer of a Multi-Head Attention followed by another layer of Feed Forward Neural Network. The decoder, on the other hand, has an extra Masked Multi-Head Attention.<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-20-01-32.png)<br>


**Encoder**: The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection around each ofthe two sub-layers, followed by layer normalization. That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)),where Sublayer(x) is the function implemented by the sub-layeritself. To facilitate these residual connections, all sub-layers in the model, as well as the embeddinglayers, produce outputs of dimension dmodel = 512.<br>


**Decoder**: The decoder is also composed of a stack of N = 6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.<br>

The encoder and decoder blocks are actually multiple identical encoders and decoders stacked on top of each other. Both the encoder stack and the decoder stack have the same number of units.The number of encoder and decoder units is a hyperparameter <br>

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-20-03-14.png)<br>

Let’s see how this setup of the encoder and the decoder stack works:<br>

* The word embeddings of the input sequence are passed to the first encoder <br>
* These are then transformed and propagated to the next encoder<br>
* The output from the last encoder in the encoder-stack is passed to all the decoders in the decoder-stack as shown in the figure below:<br>

An important thing to note here – in addition to the self-attention and feed-forward layers, the decoders also have one more layer of Encoder-Decoder Attention layer. This helps the decoder focus on the appropriate parts of the input sequence.<br>

## 2.2 Understanding self attention: <br>
Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-22-31-11.png)<br>
Take a look at the above image. Can you figure out what the term “it” in this sentence refers to?<br>

Is it referring to the street or to the animal? It’s a simple question for us but not for an algorithm. When the model is processing the word “it”, self-attention tries to associate “it” with “animal” in the same sentence.<br>

Self-attention allows the model to look at the other words in the input sequence to get a better understanding of a certain word in the sequence.<br>

Self-attention is computed not once but multiple times in the Transformer’s architecture, in parallel and independently. It is therefore referred to as Multi-head Attention. The outputs are concatenated and linearly transformed as shown in the figure below:
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-22-47-53.png)<br>

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions.<br>

## 2.3 Limitations of the Transformer:<br>
Attention can only deal with fixed-length text strings. The text has to be split into a certain number of segments or chunks before being fed into the system as input
This chunking of text causes context fragmentation. For example, if a sentence is split from the middle, then a significant amount of context is lost. In other words, the text is split without respecting the sentence or any other semantic boundary <br>

But,undoutedly transformer inspired BERT and all the following breakthroughs in NLP.

# 3. Understanding BERT (Bidirectional Encoder Representations from Transformers): <br>
BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications.<br>

BERT is a **deeply bidirectional** model. Bidirectional means that BERT learns information from both the left and the right side of a token’s context during the training phase.<br>

The bidirectionality of a model is important for truly understanding the meaning of a language. Let’s see an example to illustrate this. There are two sentences in this example and both of them involve the word “bank”:<br>

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/sent_context.png)<br>

If we try to predict the nature of the word “bank” by only taking either the left or the right context, then we will be making an error in at least one of the two given examples.One way to deal with this is to consider both the left and the right context before making a prediction. That’s exactly what BERT does. <br>

## 3.1 How Does BERT Work? <br>
### 3.1.1 BERT’s Architecture: <br>

The BERT architecture builds on top of Transformer. We currently have two variants available:<br>

* BERT Base: 12 layers (transformer blocks), 12 attention heads, and 110 million parameters
* BERT Large: 24 layers (transformer blocks), 16 attention heads and, 340 million parameters

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/bert_encoder.png/)<br>

All of these Transformer layers are Encoder-only blocks.<br>

Now that we know the overall architecture of BERT, let’s see what kind of text processing steps are required before we get to the model building phase.<br>

## 3.2 Text processing for BERT:<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/bert_emnedding.png)<br>
For bert,every input embedding is a combination of 3 embeddings:<br>

**Position Embeddings**: BERT learns and uses positional embeddings to express the position of words in a sentence. These are added to overcome the limitation of Transformer which, unlike an RNN, is not able to capture “sequence” or “order” information<br>
**Segment Embeddings**: BERT can also take sentence pairs as inputs for tasks (Question-Answering). That’s why it learns a unique embedding for the first and the second sentences to help the model distinguish between them. In the above example, all the tokens marked as EA belong to sentence A (and similarly for EB)<br>
**Token Embeddings**: These are the embeddings learned for the specific token from the WordPiece token vocabulary<br>

For a given token, its input representation is constructed by summing the corresponding token, segment, and position embeddings.Such a comprehensive embedding scheme contains a lot of useful information for the model.These combinations of preprocessing steps make BERT so versatile.<br>

When training language models, there is a challenge of defining a prediction goal. Many models predict the next word in a sequence, a directional approach which inherently limits context learning. To overcome this challenge, BERT uses two training strategies:<br>

### 1. Masked LM (MLM): <br>
Before feeding word sequences into BERT, 15% of the words in each sequence are replaced with a [MASK] token. The model then attempts to predict the original value of the masked words, based on the context provided by the other, non-masked, words in the sequence. In technical terms, the prediction of the output words requires:
1. Adding a classification layer on top of the encoder output.<br>
2. Multiplying the output vectors by the embedding matrix, transforming them into the vocabulary dimension.<br>
3. Calculating the probability of each word in the vocabulary with softmax.<br>

![](https://miro.medium.com/max/986/0*ViwaI3Vvbnd-CJSQ.png)<br>
The BERT loss function takes into consideration only the prediction of the masked values and ignores the prediction of the non-masked words. As a consequence, the model converges slower than directional models, a characteristic which is offset by its increased context awareness.<br>

### 2. Next Sentence Prediction (NSP): <br>
In the BERT training process, the model receives pairs of sentences as input and learns to predict if the second sentence in the pair is the subsequent sentence in the original document. During training, 50% of the inputs are a pair in which the second sentence is the subsequent sentence in the original document, while in the other 50% a random sentence from the corpus is chosen as the second sentence. The assumption is that the random sentence will be disconnected from the first sentence.<br>
To help the model distinguish between the two sentences in training, the input is processed in the following way before entering the model:<br>
1. A [CLS] token is inserted at the beginning of the first sentence and a [SEP] token is inserted at the end of each sentence.
2. A sentence embedding indicating Sentence A or Sentence B is added to each token. Sentence embeddings are similar in concept to token embeddings with a vocabulary of 2.
3. A positional embedding is added to each token to indicate its position in the sequence. The concept and implementation of positional embedding are presented in the Transformer paper.
![](https://miro.medium.com/max/1321/0*m_kXt3uqZH9e7H4w.png)<br>

To predict if the second sentence is indeed connected to the first, the following steps are performed:
1. The entire input sequence goes through the Transformer model.
2. The output of the [CLS] token is transformed into a 2×1 shaped vector, using a simple classification layer (learned matrices of weights and biases).
3. Calculating the probability of IsNextSequence with softmax.<br>

When training the BERT model, Masked LM and Next Sentence Prediction are trained together, with the goal of minimizing the combined loss function of the two strategies.

## 3.3 How to use BERT (Fine-tuning):
Using BERT for a specific task is relatively straightforward:<br>
BERT can be used for a wide variety of language tasks, while only adding a small layer to the core model:<br>
1. Classification tasks such as sentiment analysis are done similarly to Next Sentence classification, by adding a classification layer on top of the Transformer output for the [CLS] token.<br>
2. In Question Answering tasks (e.g. SQuAD v1.1), the software receives a question regarding a text sequence and is required to mark the answer in the sequence. Using BERT, a Q&A model can be trained by learning two extra vectors that mark the beginning and the end of the answer.<br>
3. In Named Entity Recognition (NER), the software receives a text sequence and is required to mark the various types of entities (Person, Organization, Date, etc) that appear in the text. Using BERT, a NER model can be trained by feeding the output vector of each token into a classification layer that predicts the NER label.<br>

## 3.4 Takeaways:
1. Model size matters, even at huge scale. BERT_large, with 345 million parameters, is the largest model of its kind. It is demonstrably superior on small-scale tasks to BERT_base, which uses the same architecture with “only” 110 million parameters.<br>
2. With enough training data, more training steps == higher accuracy. For instance, on the MNLI task, the BERT_base accuracy improves by 1.0% when trained on 1M steps (128,000 words batch size) compared to 500K steps with the same batch size.<br>
3. BERT’s bidirectional approach (MLM) converges slower than left-to-right approaches (because only 15% of words are predicted in each batch) but bidirectional training still outperforms left-to-right training after a small number of pre-training steps.<br>

![](https://miro.medium.com/max/1773/0*KONsqvDohE7ytu_E.png)

## 3.5 Compute considerations (training and applying):<br>
![](https://miro.medium.com/max/875/1*LgbpLsRUGbtTPmMSUO2Drw.png)<br>

## 3.6 Tokenization:<br>
Subword tokenization is a technique used in natural language processing (NLP) to segment words into smaller units called subwords. In this approach, words are broken down into their constituent subwords, which can be useful in many NLP tasks such as machine translation, language modeling, and sentiment analysis.

The main advantage of subword tokenization is that it can handle rare and unknown words by representing them as a combination of common subwords. This can improve the robustness of NLP models, particularly when dealing with languages that have complex morphology or large vocabularies.

There are several techniques for subword tokenization, including Byte Pair Encoding (BPE), WordPiece, and Unigram Language Model (ULM). These techniques differ in the way they segment words into subwords and in the granularity of the resulting subwords.

Subword tokenization has become increasingly important in recent years with the rise of deep learning models that require fixed-size inputs. Instead of using a fixed vocabulary of individual words, subword tokenization can break down words into variable-length subwords, which can improve the performance of NLP models on rare and out-of-vocabulary words.






In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm

# Define the corpus
with open('corpus.txt', 'w') as f:
    f.write("The quick brown fox jumps over the lazy dog\n")
    f.write("The lazy dog slept over the veranda\n")

# Train the BPE tokenizer
num_symbols = 100
spm.SentencePieceTrainer.train(
    input='corpus.txt', 
    model_prefix='bpe', 
    vocab_size=num_symbols,
    model_type='bpe'
)

# Load the trained BPE tokenizer
tokenizer = spm.SentencePieceProcessor()
tokenizer.load('bpe.model')

# Encode a sentence using the BPE tokenizer
sentence = "The lazy dog jumped over the fence"
encoded_sentence = tokenizer.encode_as_pieces(sentence)
print(encoded_sentence)

# Decode a sequence of BPE tokens
decoded_sentence = tokenizer.decode_pieces(encoded_sentence)
print(decoded_sentence)

['▁The', '▁lazy', '▁dog', '▁ju', 'mp', 'e', 'd', '▁over', '▁the', '▁f', 'e', 'n', 'c', 'e']
The lazy dog jumped over the fence


## 3.7 Sequence Classification:<br>

In [ ]:
!pip install transformers datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.8 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import datasets

In [ ]:
train_data, test_data = datasets.load_dataset('imdb', split=['train[:10%]', 'test[:40%]'])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# preprocess data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def preprocess_data(example):
    encoded = tokenizer.encode_plus(
        example['text'], 
        max_length=512,
        truncation=True, 
        padding='max_length',
        add_special_tokens=True, 
        return_attention_mask=True
    )
    return {"input_ids":encoded['input_ids'], "attention_mask": encoded['attention_mask'], "label": example['label']}

In [ ]:
train_dataset = train_data.map(lambda example: preprocess_data(example))
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset = test_data.map(lambda example: preprocess_data(example))
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# define model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to('cuda')

# define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# define data loader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
from tqdm.notebook import tqdm
# define training and evaluation functions
def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        label = batch['label'].to('cuda')
        optimizer.zero_grad()
        output = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = output.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader)

def evaluate(model, test_loader):
    model.eval()
    total_correct = 0
    with torch.no_grad():
        for batch in tqdm(test_loader):
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')
            label = batch['label'].to('cuda')
            output = model(input_ids, attention_mask=attention_mask, labels=label)
            logits = output.logits
            predictions = torch.argmax(logits, dim=-1)
            total_correct += torch.sum(predictions == label).item()
    accuracy = total_correct / len(test_dataset)
    return accuracy

In [ ]:
# train model
num_epochs = 1
for epoch in range(num_epochs):
    # train_loss = train(model, train_loader, optimizer)
    test_accuracy = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.3f}, Test Accuracy: {test_accuracy:.3f}')

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch 1/1, Train Loss: 0.031, Test Accuracy: 1.000


In [ ]:
# train model
num_epochs = 1
for epoch in range(num_epochs):
    # train_loss = train(model, train_loader, optimizer)
    test_accuracy = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.3f}, Test Accuracy: {test_accuracy:.3f}')

  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch 1/1, Train Loss: 0.031, Test Accuracy: 1.000


<a id="1_9"></a>

<br><h3 style="font-family: Georgia; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;">SUMMARY</h3>

---

<p style="font-family: Georgia;">In this chapter, you saw how to approach different NLP tasks using 🤗 Transformers. You also saw how to search for and use models in the <b><a href="https://huggingface.co/models">Hub</a></b></p>

<p style="font-family: Georgia;">We discussed how Transformer models work at a high level, and talked about the importance of transfer learning and fine-tuning. A key aspect is that you can use the full architecture or only the encoder or decoder, depending on what kind of task you aim to solve. The following table summarizes this:</p><br>

---

<br><table style="font-family: Georgia; text-align:center;">
    <thead>
        <tr>
            <th>Model</th>
            <th>Examples</th>
            <th>Tasks</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Encoder</td>
            <td>ALBERT, BERT, DistilBERT, ELECTRA, RoBERTa</td>
            <td>Sentence classification, named entity recognition, extractive question answering</td>
        </tr>
        <tr>
            <td>Decoder</td>
            <td>CTRL, GPT, GPT-2, Transformer XL</td>
            <td>Text generation</td></tr>
        <tr>
            <td>Encoder-decoder</td>
            <td>BART, T5, Marian, mBART</td>
            <td>Summarization, translation, generative question answering</td>
        </tr>
    </tbody>
</table><br>

---

# Task: Bangla text generation using GPT-2

## Rough experimental work on English and Bangla GPT-2 models.

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.7 MB/s eta 0:00:00


Trying out the english GPT-2.

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, I'm writing a new language for you. But first, I'd like to tell you about the language itself"},
 {'generated_text': "Hello, I'm a language model, and I'm trying to be as expressive as possible. In order to be expressive, it is necessary to know"},
 {'generated_text': "Hello, I'm a language model, so I don't get much of a license anymore, but I'm probably more familiar with other languages on that"},
 {'generated_text': "Hello, I'm a language model, a functional model... It's not me, it's me!\n\nI won't bore you with how"},
 {'generated_text': "Hello, I'm a language model, not an object model.\n\nIn a nutshell, I need to give language model a set of properties that"}]

**Bengali GPT-2** from https://huggingface.co/flax-community/gpt2-bengali.

In [ ]:
gpt2_bengali = pipeline('text-generation',model="flax-community/gpt2-bengali", tokenizer='flax-community/gpt2-bengali')

The Bengali GPT-2 finetuned model on Bangla songs. 

In [ ]:
gpt2_bengali("তুমি যদি চাও তবে", max_length=100, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'তুমি যদি চাও তবে একটি ফুল হাতে নিয়ে আলোকিত করে গুঁজে দিতে পার, তবুও ভালোবাসি। তুমি বলেছিলে বিধ্বংসী মহামারি বিশ্ব দেয় দুটি স্বপ্নকুঠি আর'},
 {'generated_text': 'তুমি যদি চাও তবে শূন্যতাও কিছুটা আঁকড়ে ধরো। মাতার কাছ হতে, তার নিজের যা কিছু আছে; ভালোবাসার বৈশিষ্ট্য যাতে তোমরা দেখতে পাও; জীবন ও সত্য ও সু'},
 {'generated_text': 'তুমি যদি চাও তবে তারা দুজনে নামাযে দাঁড়াবে না, তোমার নামায একাকী ও নিশ্চিত না হওয়া পর্যন্ত তারা বসবে। -সূরা হজ, আয়াতঃ ২০ বিধবা মা সন্তান ভূমিষ্ঠ'},
 {'generated_text': 'তুমি যদি চাও তবে বিজ্ঞাপনলগ ইনআপনার বিজ্ঞাপন দিনসকল বিজ্ঞাপনঘর ও বাগানের সামগ্রীআসবাবপত্র মধ্যে মোহাম্মদপুর, ঢাকাbest offer m/s 2006মেরামত কিট সদস্য এর মাধ্যমে ব'},
 {'generated_text': "তুমি যদি চাও তবে ব্রিটিশ ভারত ১৯৪৭ সালের ২৭ নভেম্বর বিছানায় শোওয়ানো হবে।' (পৃ. ২৬)। 'আমি মুসলমান, ভারতের সম্পত্তি নই অথচ ধর্মভিত্তিক সাম্প্রদায়িক সম্প্"}]

In [ ]:
singer = pipeline('text-generation',model="khalidsaifullaah/bengali-lyricist-gpt2", tokenizer='khalidsaifullaah/bengali-lyricist-gpt2')

In [ ]:
singer("তুমি যদি চাও তবে", max_length=100, num_return_sequences=5)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'তুমি যদি চাও তবে আমাকেও কোলে নিয়ে সোজা চলে যেতে পারো দূরে তোমাকেই ধরে আছি যতদিন বাঁচি তোমারই সেবা করে যাবো আমি তুমি আমাদেরই সেই পরিচয় দেবে এসে আম'},
 {'generated_text': 'তুমি যদি চাও তবে সাগর যেমন কর্ণফুলী নদী হয় যদি চাও তবে মরুভূমি, তবু আমাকে নাহয় ধরো মরুর নীল তবু মেঘের আঁচলে নীল হতে চাইতে চাইতো পৃথিবীটা যদি চাও নীলগাই'},
 {'generated_text': 'তুমি যদি চাও তবে বন্ধু তুমি চাঁদেরই হাট বলে দাও তোমাকে আমার কত প্রিয় এই শহর বলে দাও তোমাকে আমার কত প্রিয় এই শহর আমার আমার শহর দিয়েছো তুমি আমারি ভীড় এই শহর'},
 {'generated_text': 'তুমি যদি চাও তবে দুটি শিকল বন্ধুর কাছে সঁপিয়ে দিও আমি বিশ্ব কবি। ভালবেসে যত পাবে তোমায় আর কিছু চাওয়ার নেই আমার তবু হায়! নিঃস্ব তুমি আছ বান্ধবী'},
 {'generated_text': 'তুমি যদি চাও তবে হে আমার প্রার্থনায়, তুমি শান্ত হয়ে, সব তালিম বন্ধ করে, বিছানায় আমার সঙ্গ দিয়ো, তোমার জন্য একটি সুন্দর সূর্য তুলে দেব, একটুও ক্ষত'}]

In [ ]:
gpt2_bengali_lyricist = pipeline('text-generation',model="khalidsaifullaah/bengali-lyricist-gpt2", tokenizer='khalidsaifullaah/bengali-lyricist-gpt2')

This model is a fine-tuned version of [`flax-community/gpt2-bengali`](https://huggingface.co/khalidsaifullaah/bengali-lyricist-gpt2?text=%E0%A6%9C%E0%A7%80%E0%A6%AC%E0%A6%A8%E0%A7%87%E0%A6%B0+%E0%A6%AE%E0%A6%BE%E0%A6%A8%E0%A7%87) on the [Bengali Song Lyrics](https://www.kaggle.com/shakirulhasan/bangla-song-lyrics) dataset from Kaggle.

In [ ]:
gpt2_bengali_lyricist("তুমি যদি চাও তবে", max_length=100, num_return_sequences=5)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'তুমি যদি চাও তবে ফিরাব তোমায় আরো বলবো যে কষ্ট তোমার অনেক দিনের পুরানো সে কথা স্বপ্ন জুড়ে গেলে কষ্ট আমি সইতে পারবো না বলে সৃতির সবুজ সাগরে দিয়েছি'},
 {'generated_text': 'তুমি যদি চাও তবে শোন বিদ্রোহী কবির কবিতা আজ বাজিয়ে বিদায় দিয়ে এসো আকাশে দীর্ঘশ্বাসের আড়ালে তোমারই সৃষ্টি শুধু ছিঁড়ে ফেল আমার প্রাণ তুম'},
 {'generated_text': 'তুমি যদি চাও তবে জীবনের সব আশার বাসনা যাব ত্যাগের অতলে ব’সে ব’লো না ঝরে ব’লো না ঝরো না।।আমি করিনা আপন এ কাজ আপনি করেনা আঁখি মেলে সে চায় দ্বিধাহীন চো'},
 {'generated_text': 'তুমি যদি চাও তবে প্রেরণা দা’দা’র হাতেই সাত্ত্বিক স্রোতে আমার সাত্ত্বিক বোনাস সাত্ত্বিক তুমি যদি চাও তবে আমায় সাথী করে নাও জিয়া হায়'},
 {'generated_text': 'তুমি যদি চাও তবে রাতের আঁচল বুকে জড়িয়ে দেখতে পারো সব পোড়া ঘর।।কাল সকালে উঠে মাথার উপর মেঘ জমেছে ব্যাকুলতার প্রভাতে মনে পড়ে আমার আজ তোমাকে, আমি কাঁদছি ন'}]

**Bangla GPT-2** from https://huggingface.co/ghosh-r/bangla-gpt2.

In [ ]:
gpt2_bengali2 = pipeline('text-generation',model="ghosh-r/bangla-gpt2", tokenizer='ghosh-r/bangla-gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
gpt2_bengali2("তুমি যদি চাও তবে", max_length=100, num_return_sequences=5)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'তুমি যদি চাও তবে প্রতিপক্ষ মানসিক ভাবে তার সিদ্ধান্ত পরিবর্তন করতে পারে। আপনার হৃদয়ের হাসি দান করুক। এই সব ভাল। বাস্তব সময় চেষ্টা থাকলে বাউন্স এবং ব'},
 {'generated_text': 'তুমি যদি চাও তবে তুমি আবার ফিরে আসবে রাতের দিকে একটু বিরতি নিয়ে। এই ফেরে আবার আমার সাথেই ফিরে আসলে তুমি।\nমাত্র চার দিন আগের কথা। একটা বিষাক্ত ছোবল। ম'},
 {'generated_text': 'তুমি যদি চাও তবে তুমি যা ইচ্ছা করো। তোমার হাতে দিবো। তাই তোমাকে মাসিক দিবো আমি। আর সুখের স্বামী না স্ত্রী। যাকে স্ত্রী’ হিসেবে মানি না। আর ত'},
 {'generated_text': 'তুমি যদি চাও তবে তুমিই সালাম জানাও। মানুষ হিসাবে আমি নিজের সম্মানের দিকটা বিবেচনা করে সালাম দিই। কিন্তু প্রত্যেক মানুষই সুস্বাগত জা'},
 {'generated_text': 'তুমি যদি চাও তবে তারা আমার কাছে আসবে না; তাদের পশর- প্রয়োজনে আমার কাছে আসবে না, যার হিসাব পাওয়া গেছে তখন আমি তাদেরকে ডাকব না। তোমরা উভয়ে নিজদের সাথে সুসম্'}]

Let's use `saiful9379/Bangla_GPT2` as our generator model. It was trained using the **Bangla Newspaper dataset**. They used Prothom Alo $250$MB data for GPT-2 model training with a vocab size of $50$k. Visit https://github.com/saiful9379/Bangla_GPT2 to learn more about this GPT-2 model.



In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("saiful9379/Bangla_GPT2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("saiful9379/Bangla_GPT2", pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at saiful9379/Bangla_GPT2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## Greedy Search
Greedy search simply selects the word with the highest probability as its next word: $w_t = argmax_{w}P(w | w_{1:t-1})$ at each timestep $t$. The following sketch shows greedy search. 

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

Starting from the word $\text{"The"}$, the algorithm 
greedily chooses the next word of highest probability $\text{"nice"}$ and so on, so that the final generated word sequence is $\text{"The", "nice", "woman"}$ having an overall probability of $0.5 \times 0.4 = 0.2$.

In the following we will generate word sequences using GPT2 on the context $(\text{"বহুল", "আলোচিত", "দশম", "জাতীয়", "সংসদ"})$. Let's see how greedy search can be used in `transformers` as follows:

In [ ]:
text = "বহুল আলোচিত দশম জাতীয় সংসদ"
input_ids = tokenizer.encode(text, return_tensors='tf')

In [ ]:
greedy_output = model.generate(input_ids, max_length=200)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের পর বিএনপির নেতৃত্বাধীন ১৮-দলীয় জোটের বিজয়ী দল আওয়ামী লীগের নেতৃত্বাধীন ১৮-দলীয় জোটের বিজয়কে স্বাগত জানিয়েছে। এই নির্বাচনের ফলাফল কী দাঁড়াবে. তা নিয়ে সংশয় দেখা দিয়েছে। এই নির্বাচনের ফলাফল কী দাঁড়াবে. তা নিয়ে সংশয় দেখা দ


The major drawback of greedy search though is that it misses high probability words hidden behind a low probability word as can be seen in our sketch above:

The word $\text{"has"}$ with its high conditional probability of $0.9$ is hidden behind the word $\text{"dog"}$, which has only the second-highest conditional probability, so that greedy search misses the word sequence $\text{"The"}, \text{"dog"}, \text{"has"}$.

Thankfully, we have beam search to alleviate this problem!

## Beam search

Beam search reduces the risk of missing hidden high probability word sequences by keeping the most likely `num_beams` of hypotheses at each time step and eventually choosing the hypothesis that has the overall highest probability. Let's illustrate with `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

At time step $1$, besides the most likely hypothesis $\text{"The", "nice"}$, beam search also keeps track of the second most likely one $\text{"The", "dog"}$. At time step $2$, beam search finds that the word sequence $\text{"The", "dog", "has"}$ has with $0.36$ a higher probability than $\text{"The", "nice", "woman"}$, which has $0.2$. Great, it has found the most likely word sequence in our toy example! 

Beam search will always find an output sequence with higher probability than greedy search, but is not guaranteed to find the most likely output. 

Let's see how beam search can be used in `transformers`. We set `num_beams > 1` and `early_stopping=True` so that generation is finished when all beam hypotheses reached the EOS token.

In [ ]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=200, 
    num_beams=5, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের প্রাক্কালে জাতীয় পার্টির চেয়ারম্যান হুসেইন মুহম্মদ এরশাদ নির্বাচনকালীন সরকারের প্রধান হিসেবে দায়িত্ব পালন করেছিলেন। সেই সরকারের আমলে জাতীয় পার্টির চেয়ারম্যান হুসেইন মুহম্মদ এরশাদ নির্বাচিত হয়েছিলেন। জাতীয় পার্টির চে�


While the result is arguably more fluent, the output still includes repetitions of the same word sequences.  
A simple remedy is to introduce *n-grams* (*a.k.a* word sequences of $n$ words) penalties as introduced by [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) and [Klein et al. (2017)](https://arxiv.org/abs/1701.02810). The most common *n-grams* penalty makes sure that no *n-gram* appears twice by manually setting the probability of next words that could create an already seen *n-gram* to $0$.

Let's try it out by setting `no_repeat_ngram_size=2` so that no *2-gram* appears twice:

In [ ]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের তফসিল ঘোষণার পর থেকেই বিভিন্ন স্থানে সহিংসতা ও সন্ত্রাসী কর্মকাণ্ড পরিচালনা করা হচ্ছে। সাম্প্রতিক হরতাল-অবরোধে যানবাহন ভাঙচুর ও আগুন লাগাসহ বেশ কিছু ঘটনা ঘটেছে বলে গণমাধ্যমে প্রকাশ। তবে সেগু
লোর পেছনে কোনো কারণ ছাড়াই আইনশৃঙ্খলা রক্ষা. রাষ্ট্রপতি ও সাংবিধ


Nice, that looks much better! We can see that the repetition does not appear anymore. Nevertheless, *n-gram* penalties have to be used with care. An article generated about the city *New York* should not use a *2-gram* penalty or otherwise, the name of the city would only appear once in the whole text!

Another important feature about beam search is that we can compare the top beams after generation and choose the generated beam that fits our purpose best. 

In `transformers`, we simply set the parameter `num_return_sequences` to the number of highest scoring beams that should be returned. Make sure though that `num_return_sequences <= num_beams`!

In [ ]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের তফসিল ঘোষণার পর থেকেই বিভিন্ন স্থানে সহিংসতা ও সন্ত্রাসী কর্মকাণ্ড পরিচালনা করা হচ্ছে। সাম্প্রতিক হরতাল-অবরোধে যানবাহন ভাঙচুর ও আগুন লাগাসহ বেশ কিছু ঘটনা ঘটেছে বলে গণমাধ্যমে প্রকাশ। তবে সেগু
লোর পেছনে কোনো কারণ ছাড়াই আইনশৃঙ্খলা রক্ষা. রাষ্ট্রপতি ও সাংবিধ
1: বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের তফসিল ঘোষণার পর থেকেই বিভিন্ন স্থানে সহিংসতা ও সন্ত্রাসী কর্মকাণ্ড পরিচালনা করা হচ্ছে। সাম্প্রতিক হরতাল-অবরোধে যানবাহন ভাঙচুর ও আগুন লাগাসহ বেশ কিছু ঘটনা ঘটেছে বলে গণমাধ্যমে প্রকাশ। তবে সেগু
লোর পেছনে কোনো কারণ ছাড়াই আইনশৃঙ্খলা রক্ষা. রাজনৈতিকভাবে ব্যা
2: বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের তফসিল ঘোষণার পর থেকেই বিভিন্ন স্থানে সহিংসতা ও সন্ত্রাসী কর্মকাণ্ড পরিচালনা করা হচ্ছে। সাম্প্রতিক হরতাল-অবরোধে যানবাহন ভাঙচুর ও আগুন লাগাসহ বেশ কিছু ঘটনা ঘটেছে বলে গণমাধ্যমে প্রকাশ। তবে সেগু
লোর পেছনে কোনো কারণ ছাড়াই আইনশৃঙ্খলা রক্ষা. রাজনৈতিকভাবে ব্যক্
3: বহু

As can be seen, the five beam hypotheses are only marginally different to each other - which should not be too surprising when using only 5 beams.

In open-ended generation, a couple of reasons have recently been brought forward why beam search might not be the best possible option:

- Beam search can work very well in tasks where the length of the desired generation is more or less predictable as in machine translation or summarization - see [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) and [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). But this is not the case for open-ended generation where the desired output length can vary greatly, e.g. dialog and story generation.

- We have seen that beam search heavily suffers from repetitive generation. This is especially hard to control with *n-gram*- or other penalties in story generation since finding a good trade-off between forced "no-repetition" and repeating cycles of identical *n-grams* requires a lot of finetuning.

- As argued in [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751), high quality human language does not follow a distribution of high probability next words. In other words, as humans, we want generated text to surprise us and not to be boring/predictable. The authors show this nicely by plotting the probability, a model would give to human text vs. what beam search does.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)

## Sampling

In its most basic form, sampling means randomly picking the next word $w_t$ according to its conditional probability distribution:

$$w_t \sim P(w|w_{1:t-1})$$

Taking the example from above, the following graphic visualizes language generation when sampling.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

It becomes obvious that language generation using sampling is not *deterministic* anymore. The word 
$\text{"car"}$ is sampled from the conditioned probability distribution $P(w | \text{"The"})$, followed by sampling $\text{"drives"}$ from $P(w | \text{"The"}, \text{"car"})$.

In `transformers`, we set `do_sample=True` and deactivate *Top-$k$* sampling (more on this later) via `top_k=0`. In the following, we will fix `random_seed=0` for illustration purposes. Feel free to change the `random_seed` to play around with the model.

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(42)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=200, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচন আওয়ামী লীগ নির্বাচন কমিশনকে বেকায়দায় ফেলতে পারবে বলে মনে হচ্ছে। কিন্তু এ ধরনের নির্বাচন আয়োঞজনেরা হতাশাব্যঞ্জক অনুকূল পরিবেশ তৈরি করেছেন এবং সেটা সহিংসতায় রূপ নিয়েছে। বিশেষ করে সিইসি বলেছেন. ‘জাতীয় নির্বাচনকে আমরা বৈধতা দিতে পেরেছি।’ (প্রথম আলো.


Interesting! The text seems alright - but when taking a closer look, it is not very coherent. the *3-grams* *new hand sense* and *local batte harness* are very weird and don't sound like they were written by a human. That is the big problem when sampling word sequences: The models often generate incoherent gibberish, *cf.* [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751).

A trick is to make the distribution $P(w|w_{1:t-1})$ sharper (increasing the likelihood of high probability words and decreasing the likelihood of low probability words) by lowering the so-called `temperature` of the [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max). 

An illustration of applying temperature to our example from above could look as follows.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

The conditional next word distribution of step $t=1$ becomes much sharper leaving almost no chance for word $\text{"car"}$ to be selected.


Let's see how we can cool down the distribution in the library by setting `temperature=0.7`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=200, 
    top_k=0, 
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচন সম্পন্ন করা নিয়ে বিএনপির বিজয়ের দিন স্পষ্টভাবে আলোচনার সুযোগ করে দিয়েছে। আওয়ামী লীগ সরকারের মেয়াদের শেষ বছরে এসে বিএনপিকে নির্বাচনে আনতে পারলে তা ভালো হতো। তবে এতে কোনো সন্দেহ নেই. নির্বাচনের মাধ্যমে আওয়ামী লীগ আরও পাঁচ বছর ক্ষমতায় থাকতে পারত। আমাদে


OK. There are less weird n-grams and the output is a bit more coherent now! While applying temperature can make a distribution less random, in its limit, when setting `temperature` $ \to 0$, temperature scaled sampling becomes equal to greedy decoding and will suffer from the same problems as before. 



## Top-$k$ Sampling

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) introduced a simple, but very powerful sampling scheme, called ***Top-K*** sampling. In *Top-K* sampling, the *K* most likely next words are filtered and the probability mass is redistributed among only those *K* next words. 
GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation. 

We extend the range of words used for both sampling steps in the example above from 3 words to 10 words to better illustrate *Top-K* sampling.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Having set $K = 6$, in both sampling steps we limit our sampling pool to 6 words. While the 6 most likely words, defined as $V_{\text{top-K}}$ encompass only *ca.* two-thirds of the whole probability mass in the first step, it includes almost all of the probability mass in the second step. Nevertheless, we see that it successfully eliminates the rather weird candidates $\text{"not", "the", "small", "told"}$ 
in the second sampling step.


Let's see how *Top-K* can be used in the library by setting `top_k=50`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=200, 
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচন সম্পন্ন করা না হলে সে নির্বাচন যেমন অবাধই হতো। এখনো হলো। সংবিধান সংশোধন করা প্রথম থেকে শেষ ২৪ জানুয়ারি পর্যন্ত নির্বাচনটি অবাধ ও শান্তিপূর্ণ হবে কি না. তা তারা ভেবে দেখতে পারে। প্রধান নির্বাচন কমিশনার ক
তে পারেননি বলেই আজ আমাদের ভুল হচ্ছে। তবে এসব দাবি নিয়ে দ


Not bad at all! The text is arguably the most *human-sounding* text so far. 
One concern though with *Top-K* sampling is that it does not dynamically adapt the number of words that are filtered from the next word probability distribution $P(w|w_{1:t-1})$.
This can be problematic as some words might be sampled from a very sharp distribution (distribution on the right in the graph above), whereas others from a much more flat distribution (distribution on the left in the graph above). 

In step $t=1$, *Top-$k$* eliminates the possibility to 
sample $\text{"people", "big", "house", "cat"}$, which seem like reasonable candidates. On the other hand, in step $t=2$ the method includes the arguably ill-fitted words $\text{"down", "a"}$ in the sample pool of words. Thus, limiting the sample pool to a fixed size *K* could endanger the model to produce gibberish for sharp distributions and limit the model's creativity for flat distribution.
This intuition led [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) to create ***Top-$p$*** or ***nucleus*** sampling. 



## Top-$p$ (nucleus) Sampling

Instead of sampling only from the most likely *K* words, in *Top-p* sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability *p*. The probability mass is then redistributed among this set of words. This way, the size of the set of words (*a.k.a* the number of words in the set) can dynamically increase and decrease according to the next word's probability distribution. Ok, that was very wordy, let's visualize.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Having set $p=0.92$, *Top-p* sampling picks the *minimum* number of words to exceed together $p=92\%$ of the probability mass, defined as $V_{\text{top-p}}$. In the first example, this included the 9 most likely words, whereas it only has to pick the top 3 words in the second example to exceed 92%. Quite simple actually! It can be seen that it keeps a wide range of words where the next word is arguably less predictable, *e.g.* $P(w | \text{"The"})$, and only a few words when the next word seems more predictable, *e.g.* $P(w | \text{"The", "car"})$.

Alright, time to check it out in `transformers`!
We activate *Top-p* sampling by setting $0 < $ `top_p` $ < 1$:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=200, 
    top_p=0.92, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচন সম্পন্ন করা না হলে সে নির্বাচন যেমন অবাধ ও সুষ্ঠু হয়েছে. তেমনি এ নির্বাচন সংবিধান পরিপন্থী নয়। বর্তমান নির্বাচন কমিশনের বিশেষ দায়িত্ব পালন করছে তিন মাসের জন্য। কোনো প্রার্থীকে নির্বাচনে
র পরে রিটার্নিং কর্মকর্তার কাছে নিয়ে যাওয়ার সুয


Great, that sounds like it could have been written by a human. Well, maybe not quite yet. 

While in theory, *Top-$p$* seems more elegant than *Top-$k$*, both methods work  well in practice. *Top-$p$* can also be used in combination with *Top-$k$*, which can avoid very low ranked words while allowing for some dynamic selection.

Finally, to get multiple independently sampled outputs, we can *again* set the parameter `num_return_sequences > 1`: 

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=200, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচন সম্পন্ন করা না হলে সে নির্বাচন যেমন অবাধ ও সুষ্ঠু হয়েছে. তেমনি এ নির্বাচন সংবিধান পরিপন্থী নয়। বর্তমান নির্বাচন কমিশনের বিশেষ দায়িত্ব পালন করছে তিন মাসের জন্য। কোনো প্রার্থীকে নির্বাচনে
র পরে রিটার্নিং কর্মকর্তার কাছে নিয়ে যাওয়ার সুয
1: বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচনের মাধ্যমে আওয়ামী লীগের জনপ্রিয়তা বাড়ে ২০০৫ সালের ৩০ জানুয়ারি। ওই নির্বাচনের মাধ্যমে সরকার পরিবর্তন ঘটে ‘গণতন্ত্রের অভিযাত্রা’ শুরুর একটি। জনপ্রিয়তা বাড়ে. কিন্তু সম্ভবত সবার সম্মিলিত চেষ্টায় দেশের উন্নয়নের জন্য কার্যকর গণতন্ত্র বাড়ে না। নির্ব
2: বহুল আলোচিত দশম জাতীয় সংসদ নির্বাচন সম্পন্ন হওয়ার আগ পর্যন্ত সরকার নতুন সংসদের মধ্যবর্তী নির্বাচন করার বাধ্যবাধকতা সৃষ্টি করেছে। আমরা অনুমান করতে পারি. বর্তমান তফসিলের নির্বাচনের সঙ্গে সংশ্লিষ্ট থাকার পেছনে দেশের শীর্ষ স্থানীয় সরকারি স্থানীয় সরকার প্রতিষ্ঠানের ভূমিকা রয়েছে। সংবি


## **Conclusion**

As *ad-hoc* decoding methods, *top-$p$* and *top-$k$* sampling seem to produce more fluent text than traditional *greedy* - and *beam* search on open-ended language generation. 
Recently, there has been more evidence though that the apparent flaws of *greedy* and *beam* search - mainly generating repetitive word sequences - are  caused by the model (especially the way the model is trained), rather than the decoding method, *cf.* [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). Also, as demonstrated in [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492), it looks as *top-$k$* and *top-$p$* sampling also suffer from generating repetitive word sequences.

In [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf), the authors show that according to human evaluations, *beam* search can generate more fluent text than *Top-$p$* sampling, when adapting the model's training objective.

Open-ended language generation is a rapidly evolving field of research and as it is often the case there is no one-size-fits-all method here, so one has to see what works best in one's specific use case. 

That was a short introduction on how to use different decoding methods in `transformers` and recent trends in open-ended language generation.

### **Appendix**

There are a couple of additional parameters for the `generate` method that were not mentioned above. We will explain them here briefly!

- `min_length` can be used to force the model to not produce an EOS token (= not finish the sentence) before `min_length` is reached. This is used quite frequently in summarization, but can be useful in general if the user wants to have longer outputs.
- `repetition_penalty` can be used to penalize words that were already generated or belong to the context. It was first introduced by [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858) and is also used in the training objective in [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). It can be quite effective at preventing repetitions, but seems to be very sensitive to different models and use cases, *e.g.* see this [discussion](https://github.com/huggingface/transformers/pull/2303) on Github.

- `attention_mask` can be used to mask padded tokens
- `pad_token_id`, `bos_token_id`, `eos_token_id`: If the model does not have those tokens by default, the user can manually choose other token ids to represent them.

For more information please also look into the `generate` function [docstring](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate).